<a href="https://colab.research.google.com/github/CanopySimulations/canopy-python-examples/blob/project/race_eng_challenge/race_eng_challenge_google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import required libraries

In [ ]:
# You do not need to run this line everytime. Ensures the runtime supports `asyncio` async/await, and is needed on Google Colab. If the runtime is upgraded, you will be prompted to restart it, which you should do before continuing execution.
# !pip install ipython ipykernel --upgrade
!pip install -q canopy

## Define the required functions

In [166]:
import numpy as np
import canopy
import asyncio
from typing import List, Dict, Any
from copy import deepcopy
from canopy import AuthenticationData, Session, ConfigResult, StudyResult
from canopy.openapi import (
    WorksheetApi,
    StudyApi,
    ConfigApi,
    NewStudyDataSource,
    NewWorksheetDataOutline,
    WorksheetRow,
    WorksheetStudyReference,
    WorksheetRowStudy,
    StudyPostStudyRequest,
    WorksheetConfig,
    WorksheetConfigReference,
    ConfigReferenceTenant,
    WorksheetPutWorksheetRequest,
    WorksheetPostWorksheetRequest,
    GetTenantWorksheetLabelDefinitionsQueryResultLabelDefinitions,
    ConfigPostConfigRequest
)

async def authenticate_canopy_sims_api(authentication_data: AuthenticationData) -> Session:
    # Create a session with the authentication data
    session = Session(authentication_data=authentication_data)
    # Try to authenticate the session. If failed, try againwith the same canopy_sims_authentication for upto 10 times.
    for _ in range(10):
        try:
            session.authentication.authenticate()
            print("Authenticated successfully!")
            break
        except Exception as e:
            print(f"Authentication failed: {e}")
            await asyncio.sleep(1)

    return session


async def create_worksheet_and_run_study(
    session: Session,
    worksheet_name: str,
    starting_row_name: str,
    default_car_name: str,
    default_track_name: str,
    default_weather_name: str,
    default_user_maths: str=None,
    sim_version: str=None,
    sim_types: list[str] = ["DynamicLap"],
    study_type: str = "dynamicLap",
    notes: str = ""
) -> tuple[str, str]:
    """
    Creates a new worksheet with the specified name.

    Args:
        session: Authenticated Canopy session object
        worksheet_name: Name of the new worksheet
        notes: Optional notes for the new worksheet

    Returns:
        ID of the created worksheet and the ID of the created study.

    Example:
        await create_worksheet(
            session=active_session,
            worksheet_name="My New Worksheet"
        )
    """
    # Authentication and API setup
    session.authentication.authenticate()
    worksheet_api = WorksheetApi(session.async_client)

    # Create new worksheet
    result = await worksheet_api.worksheet_post_worksheet(
        tenant_id=session.authentication.tenant_id,
        worksheet_post_worksheet_request=WorksheetPostWorksheetRequest(
            name=worksheet_name,
            properties=[],
            outline= NewWorksheetDataOutline(
                rows=[],
                label_definitions=GetTenantWorksheetLabelDefinitionsQueryResultLabelDefinitions(
                    simulation_label_definitions=[],
                    config_label_definitions=[]
                )
            ),
            notes=notes
        )
    )

    if not result:
        raise ValueError("Failed to create worksheet. Please check your session and parameters.")
    
    # Get the worksheet ID from the result
    worksheet = result.worksheet
    
    # Load the config API
    session.authentication.authenticate()
    config_api = ConfigApi(session.async_client)

    if sim_version is None:
        sim_version = session.tenant_sim_version.get()

    # Placeholder for configs
    configs = []

    # Create configs if default values are provided
    if default_car_name:
        config_name= default_car_name
        config_type= "car"
        # Load the default car config
        default_config = await canopy.load_default_config(
            session=session,
            config_type=config_type,
            name=config_name,
            sim_version=sim_version
        )
        config_data = default_config.to_dict().get("data")
        # print(f"Default car config ID: {default_config.to_dict()}")
        new_config_id = await canopy.create_config(
            session=session,
            config_type=config_type,
            name=config_name,
            config_data=config_data,
            sim_version=sim_version
        )
        # print(f"Created car config ID: {car_config}")
        # Create a new worksheet config for the new config
        new_worksheet_config = WorksheetConfig(
            config_type=config_type,
            reference=WorksheetConfigReference(
                tenant=ConfigReferenceTenant(
                    tenant_id=session.authentication.tenant_id,
                    target_id=new_config_id
                )
            ),
            inherit_reference=False
        )
        configs.append(new_worksheet_config)

    if default_track_name:
        config_name= default_track_name
        config_type= "track"
        # Load the default track config
        default_config = await canopy.load_default_config(
            session=session,
            config_type=config_type,
            name=config_name,
            sim_version=sim_version
        )
        config_data = default_config.to_dict().get("data")
        # print(f"Default track config ID: {default_config.to_dict()}")
        new_config_id = await canopy.create_config(
            session=session,
            config_type=config_type,
            name=config_name,
            config_data=config_data,
            sim_version=sim_version
        )
        # print(f"Created track config ID: {track_config}")
        # Create a new worksheet config for the new config
        new_worksheet_config = WorksheetConfig(
            config_type=config_type,
            reference=WorksheetConfigReference(
                tenant=ConfigReferenceTenant(
                    tenant_id=session.authentication.tenant_id,
                    target_id=new_config_id
                )
            ),
            inherit_reference=False
        )
        configs.append(new_worksheet_config)

    if default_weather_name:
        config_name= default_weather_name
        config_type= "weather"
        # Load the default weather config
        default_config = await canopy.load_default_config(
            session=session,
            config_type=config_type,
            name=config_name,
            sim_version=sim_version
        )
        config_data = default_config.to_dict().get("data")
        # print(f"Default weather config ID: {default_config.to_dict()}")
        new_config_id = await canopy.create_config(
            session=session,
            config_type=config_type,
            name=config_name,
            config_data=config_data,
            sim_version=sim_version
        )
        # print(f"Created weather config ID: {weather_config}")
        # Create a new worksheet config for the new config
        new_worksheet_config = WorksheetConfig(
            config_type=config_type,
            reference=WorksheetConfigReference(
                tenant=ConfigReferenceTenant(
                    tenant_id=session.authentication.tenant_id,
                    target_id=new_config_id
                )
            ),
            inherit_reference=False
        )
        configs.append(new_worksheet_config)

    if default_user_maths:
        config_name = default_user_maths
        config_type = "userMaths"
        # Load the default user maths config
        default_config = await canopy.load_default_config(
            session=session,
            config_type=config_type,
            name=config_name,
            sim_version=sim_version
        )
        config_data = default_config.to_dict().get("data")
        # print(f"Default user maths config ID: {default_config.to_dict()}")
        new_config_id = await canopy.create_config(
            session=session,
            config_type=config_type,
            name=config_name,
            config_data=config_data,
            sim_version=sim_version
        )
        # print(f"Created user maths config ID: {user_maths_config}")
        # Create a new worksheet config for the new config
        new_worksheet_config = WorksheetConfig(
            config_type=config_type,
            reference=WorksheetConfigReference(
                tenant=ConfigReferenceTenant(
                    tenant_id=session.authentication.tenant_id,
                    target_id=new_config_id
                )
            ),
            inherit_reference=False
        )
        configs.append(new_worksheet_config)

    # Build study configuration
    sim_config: Dict[str, Any] = {}
    sources: List[NewStudyDataSource] = []
    
    # Process non-exploration configs
    for config in configs:
        loaded_config = await canopy.load_config(session, config.reference.tenant.target_id)
        sources.append(NewStudyDataSource(
            config_type=config.config_type,
            user_id=loaded_config.document.user_id,
            config_id=config.reference.tenant.target_id,
            name=loaded_config.document.name
        ))
        # Add to sim_config only if the config type is not exploration
        if config.config_type != "exploration":
            sim_config[config.config_type] = loaded_config.document.data

    # Create study configuration
    study = {
        'simTypes': sim_types,
        'simConfig': sim_config
    }

    # If a config of type exploration is in the new configs list, add it to the study dict
    exploration_config = next((config for config in configs if config.config_type == "exploration"), None)
    if exploration_config is not None:
        # Load the exploration config
        loaded_exploration_config = await canopy.load_config(session, exploration_config.reference.tenant.target_id)
        # Add exploration data to the study configuration
        study["exploration"] = loaded_exploration_config.document.data

    # Get the study API
    session.authentication.authenticate()
    study_api = StudyApi(session.async_client)

    # Get the tenant ID
    tenant_id = session.authentication.tenant_id

    # Create study
    study_result = await study_api.study_post_study(
        tenant_id,
        StudyPostStudyRequest(
            name=starting_row_name,
            study=study,
            is_transient=False,
            study_type=study_type,
            sources=sources,
            notes=notes,
            sim_version=sim_version
        )
    )

    # Create a new row with the new configs and study reference
    new_row = WorksheetRow(
        name=starting_row_name,
        configs=configs,
        study=WorksheetRowStudy(
            reference=WorksheetStudyReference(
                tenant_id=tenant_id,
                target_id=study_result.study_id
            )
        )
    )

    # Update worksheet outline
    worksheet.outline.rows.append(new_row)
    
    # Commit changes
    await worksheet_api.worksheet_put_worksheet(
        tenant_id,
        worksheet.worksheet_id,
        WorksheetPutWorksheetRequest(
            name=worksheet.name,
            properties=worksheet.properties,
            outline=worksheet.outline,
            notes=worksheet.notes
        )
    )

    return worksheet.worksheet_id, study_result.study_id


async def reset_worksheet(
    session: Session,
    worksheet_id: str,
    row_names: List[str]
):
    """
    Resets a worksheet to contain only specified rows while preserving label definitions.

    Args:
        session: Authenticated Canopy session object
        worksheet_id: ID of worksheet to reset
        row_names: List of row names to preserve in the worksheet

    Returns:
        Updated worksheet object

    Example:
        await reset_worksheet(
            session=active_session,
            worksheet_id="38245ca29fa94a4c863dacfef6538c8b",
            row_names=["IndyCar Hybrid Ticket 2376", "FE Gen4 Loss Map"]
        )
    """
    # Authentication and API setup
    session.authentication.authenticate()
    tenant_id = session.authentication.tenant_id
    worksheet_api = WorksheetApi(session.async_client)

    # Get current worksheet state
    worksheet_result = await worksheet_api.worksheet_get_worksheet(tenant_id, worksheet_id)
    original_worksheet = worksheet_result.worksheet

    # Create filtered outline preserving label definitions
    filtered_outline = NewWorksheetDataOutline(
        rows=[row for row in original_worksheet.outline.rows if row.name in row_names],
        label_definitions=original_worksheet.outline.label_definitions
    )

    # Prepare and execute update
    return await worksheet_api.worksheet_put_worksheet(
        tenant_id,
        worksheet_id,
        WorksheetPutWorksheetRequest(
            name=original_worksheet.name,
            properties=original_worksheet.properties,
            outline=filtered_outline,
            notes=original_worksheet.notes
        )
    )


async def get_updated_worksheet_row_after_running_study_with_existing_row_configs(
    session: Session,
    tenant_id: str,
    original_row: WorksheetRow,
    sim_version: str,
    row_suffix: str,
    sim_types: List[str] = ["DynamicLap"]
) -> WorksheetRow:
    """Process individual worksheet row and create study"""
    # Create modified row
    modified_row = WorksheetRow(
        name=f"{original_row.name} {row_suffix}",
        configs=original_row.configs,
        study=original_row.study
    )

    # Build study configuration
    sim_config: Dict[str, Any] = {}
    sources: List[NewStudyDataSource] = []

    study = {
        'simTypes': sim_types,
        'simConfig': sim_config
    }
    
    # Process non-exploration configs
    for config in modified_row.configs:
        loaded_config = await canopy.load_config(session, config.reference.tenant.target_id)
        # Add exploration to the study dict if config is exploration
        if config.config_type == "exploration":
            study["exploration"] = loaded_config.document.data
        else:
            # Add other configs to sim_config
            sim_config[config.config_type] = loaded_config.document.data

        sources.append(NewStudyDataSource(
            config_type=config.config_type,
            user_id=loaded_config.document.user_id,
            config_id=config.reference.tenant.target_id,
            name=loaded_config.document.name
        ))

    # Get study API
    study_api = StudyApi(session.async_client)

    # Create study
    study_result = await study_api.study_post_study(
        tenant_id,
        StudyPostStudyRequest(
            name=modified_row.name,
            study=study,
            is_transient=False,
            study_type="dynamicLap",
            sources=sources,
            notes="",
            sim_version=sim_version
        )
    )

    # Add study reference
    return WorksheetRow(
        name=modified_row.name,
        configs=modified_row.configs,
        study=WorksheetRowStudy(
            reference=WorksheetStudyReference(
                tenant_id=tenant_id,
                target_id=study_result.study_id
            )
        )
    )


async def get_updated_worksheet_row_after_running_study_with_given_exploration(
    session: Session,
    tenant_id: str,
    original_row: WorksheetRow,
    exploration_config: ConfigResult,
    sim_version: str,
    row_suffix: str,
    sim_types: List[str] = ["DynamicLap"]
) -> WorksheetRow:
    """Process individual worksheet row and create study"""
    # Create modified row
    modified_row = WorksheetRow(
        name=f"{original_row.name} {row_suffix}",
        configs=[c for c in original_row.configs if c.config_type != "exploration"],
        study=original_row.study
    )

    # Build study configuration
    sim_config: Dict[str, Any] = {}
    sources: List[NewStudyDataSource] = []
    
    # Process non-exploration configs
    for config in modified_row.configs:
        loaded_config = await canopy.load_config(session, config.reference.tenant.target_id)
        sim_config[config.config_type] = loaded_config.document.data
        sources.append(NewStudyDataSource(
            config_type=config.config_type,
            user_id=loaded_config.document.user_id,
            config_id=config.reference.tenant.target_id,
            name=loaded_config.document.name
        ))

    # Add exploration data
    sources.append(NewStudyDataSource(
        config_type="exploration",
        user_id=exploration_config.document.user_id,
        config_id=exploration_config.config_id,
        name=exploration_config.document.name
    ))

    # Add the exploration WorksheetConfig to the row
    exploration_worksheet_config = WorksheetConfig(
        config_type='exploration',
        reference=WorksheetConfigReference(
            tenant=ConfigReferenceTenant(
                tenant_id=tenant_id,
                target_id=exploration_config.config_id
            )
        ),
        inherit_reference=False
    )

    modified_row.configs.append(exploration_worksheet_config)

    # Get study API
    study_api = StudyApi(session.async_client)

    # Create study
    study_result = await study_api.study_post_study(
        tenant_id,
        StudyPostStudyRequest(
            name=modified_row.name,
            study={
                "simTypes": sim_types,
                "simConfig": sim_config,
                "exploration": exploration_config.document.data
            },
            is_transient=False,
            study_type="dynamicLap",
            sources=sources,
            notes="",
            sim_version=sim_version
        )
    )

    # Add study reference
    return WorksheetRow(
        name=modified_row.name,
        configs=modified_row.configs,
        study=WorksheetRowStudy(
            reference=WorksheetStudyReference(
                tenant_id=tenant_id,
                target_id=study_result.study_id
            )
        )
    )


async def get_worksheet_row_with_name_in_worksheet_with_id(
    session: Session,
    worksheet_id: str,
    worksheet_row_name: str
) -> WorksheetRow:
    """Get worksheet row by name.

    Args:
        session: Authenticated Canopy session object
        worksheet_id: ID of the target worksheet
        worksheet_row_name: Worksheet row name to retrieve

    Returns:
        WorksheetRow object if found, otherwise None
    """
    
    # Authenticate and initialize APIs
    session.authentication.authenticate()
    tenant_id = session.authentication.tenant_id
    worksheet_api = WorksheetApi(session.async_client)

    # Get worksheet data
    worksheet_result = await worksheet_api.worksheet_get_worksheet(tenant_id, worksheet_id)
    worksheet = worksheet_result.worksheet

    # Get the worksheet row by name
    worksheet_row = next((row for row in worksheet.outline.rows if row.name == worksheet_row_name), None)

    # Throw an error if the row is not found
    if not worksheet_row:
        print(f"Worksheet row '{worksheet_row_name}' not found in worksheet with ID '{worksheet_id}'.")
    
    return worksheet_row


async def check_worksheet_row_study_exists(
    worksheet_row: WorksheetRow
) -> bool:
    """Check if a worksheet row study exists.

    Args:
        worksheet_row: WorksheetRow object to check

    Returns:
        True if the study exists, False otherwise
    """
    
    # Check if the row has a study reference
    return worksheet_row.study is not None and worksheet_row.study.reference is not None


async def check_if_all_sims_in_study_succeeded(
    study: StudyResult,
) -> bool:
    """Check if all simulations in a study succeeded.

    Args:
        study: StudyResult object to check
    Returns:
        True if all simulations succeeded, False otherwise
    """
    # Check if all simulations succeeded
    return study.succeeded_simulation_count == study.simulation_count


async def get_worksheet_row_study_with_all_succeeded_sims(
    session: Session,
    worksheet_id: str,
    worksheet_row_name: str,
    sim_type: str
) -> StudyResult:
    """Get the study of a worksheet row with all succeeded simulations.

    Args:
        session: Authenticated Canopy session object
        worksheet_id: ID of the target worksheet
        worksheet_row_name: Worksheet row name to analyse
        sim_type: Simulation type to check

    Returns:
        StudyResult object if all simulations succeeded
    """
    
    # Get the worksheet row
    worksheet_row = await get_worksheet_row_with_name_in_worksheet_with_id(
        session=session,
        worksheet_id=worksheet_id,
        worksheet_row_name=worksheet_row_name
    )

    # Check if the worksheet row exists
    b_worksheet_study_exists = await check_worksheet_row_study_exists(worksheet_row)
    if b_worksheet_study_exists is False:
        raise ValueError(f"Worksheet row '{worksheet_row_name}' does not have a study reference.")
    
    # Load the study
    study_id = worksheet_row.study.reference.target_id
    study = await canopy.load_study(session=session, study_id=study_id, sim_type=sim_type, include_study_full_document=True, include_job_metadata=True, include_job_scalar_results=True, include_job_vector_metadata=True)

    # Check if all simulations succeeded
    b_all_sims_succeeded = await check_if_all_sims_in_study_succeeded(study=study)
    if b_all_sims_succeeded is False:
        raise ValueError(f"Not all simulations in study '{study_id}' succeeded. Succeeded: {study.succeeded_simulation_count}, Total: {study.simulation_count}. Please re-run with different inputs and try again.")
    
    return study


async def display_jobs_with_scalar_value_above_threshold_of_worksheet_row_study(
    session: Session,
    worksheet_id: str,
    worksheet_row_name: str,
    sim_type: str,
    scalar_name: str,
    scalar_threshold: float
):
    """Analyzes the studies in the specified worksheet row and returns a summary.

    Args:
        session: Authenticated Canopy session object
        worksheet_id: ID of the target worksheet
        worksheet_row_name: Worksheet row name to analyse
        sim_type: Simulation type to check
        scalar_name: Name of the scalar to check
        scalar_threshold: Threshold value for the scalar
        """
    
    study = await get_worksheet_row_study_with_all_succeeded_sims(
        session=session,
        worksheet_id=worksheet_id,
        worksheet_row_name=worksheet_row_name,
        sim_type=sim_type
    )
    
    # Loop through the jobs and check ìf tDynamicLapQualityMetric is above the threshold
    for job in study.jobs:
        # Check if the scalar exists in the job's scalar data
        if scalar_name not in job.scalar_data:
            raise ValueError(f"Scalar '{scalar_name}' not found in job {job.document.document_id}.")
        # Return ids of the jobs that have tDynamicLapQualityMetric above the threshold
        if job.scalar_data.get(scalar_name) > scalar_threshold:
            print(f"Job ID: {job.document.document_id} has {scalar_name}: {job.scalar_data.get(scalar_name)}")


async def get_config_with_type_in_worksheet_row(
        session: Session,
        worksheet_id: str,
        worksheet_row_name: str,
        config_type: str
)-> ConfigResult:    
    """Get a desired configuration in a worksheet row.

    Args:
        session: Authenticated Canopy session object
        worksheet_id: ID of the target worksheet
        worksheet_row_name: Worksheet row name to analyse
        """
    # Get the worksheet row
    worksheet_row = await get_worksheet_row_with_name_in_worksheet_with_id(
        session=session,
        worksheet_id=worksheet_id,
        worksheet_row_name=worksheet_row_name
    )

    # Get the config with the specified type
    config = next((c for c in worksheet_row.configs if c.config_type == config_type), None)

    if config is None:
        raise ValueError(f"Config with type '{config_type}' not found in worksheet row '{worksheet_row_name}'.")
    
    return await canopy.load_config(session, config.reference.tenant.target_id)


def get_value_at_path_in_config_data(
        config_data: Dict[str, Any],
        path: str
):
    """Get the value at the specified path in the configuration data.

    Args:
        config_data: Configuration data to check
        path: Path to check of the format 'xxx/xxxx/xxxxx...indicative of the keys in the dictionary, where each entry before a slash represents a level in the dictionary'
        """
    # Split the path into keys
    keys = path.split('/')

    # Iterate through the keys to find the value. Return None if any key is not found.
    current_data = config_data
    for key in keys:
        if key in current_data:
            current_data = current_data[key]
        else:
            return None
    # Return the value at the specified path
    return current_data


def get_config_data_derived_from_base_config_data_after_modifying_paths_with_new_values(
    base_config_data: Dict[str, Any],
    paths_and_values: Dict[str, Any],
)-> str:
    """Get a new configuration after modifying the specified paths.

    Args:
        session: Authenticated Canopy session object
        base_config_data: Base configuration data to modify
        paths_and_values: Dictionary of paths and their new values
        name_of_config_to_be_created: Name of the new configuration to be created
        sim_version: Simulation version
    """
    # Check if base_config_data is None
    if base_config_data is None:
        raise ValueError("Base configuration data is None. Please provide valid configuration data.")
    
    # Create a deep copy of the base config data
    new_config_data = deepcopy(base_config_data)

    # Loop through the paths and values and set the new values if the path is valid
    for path, value in paths_and_values.items():
        keys = path.split('/')
        current_data = new_config_data
        for key in keys[:-1]:
            if key not in current_data:
                raise ValueError(f"Path '{path}' not found in base configuration data.")
            current_data = current_data[key]
        # If value is None, remove the key
        if value is None:
            del current_data[keys[-1]]
        else:
            # Set the new value at the specified path
            if keys[-1] not in current_data:
                raise ValueError(f"Path '{path}' not found in base configuration data.")    
            current_data[keys[-1]] = value

    # Return the new configuration data
    return new_config_data
    

async def get_config_id_derived_from_base_config_data_after_modifying_paths_with_new_values(
        session: Session,
        base_config_data: Dict[str, Any],
        paths_and_values: Dict[str, Any],
        name_of_config_to_be_created: str,
        config_type: str,
        sim_version: str
) -> str:
    """Get a new configuration after modifying the specified paths.

    Args:
        session: Authenticated Canopy session object
        base_config_data: Base configuration data to modify
        paths_and_values: Dictionary of paths and their new values
        name_of_config_to_be_created: Name of the new configuration to be created
        config_type: Configuration type
        sim_version: Simulation version
    """
    # Get the base config data
    new_config_data = get_config_data_derived_from_base_config_data_after_modifying_paths_with_new_values(
        base_config_data=base_config_data,
        paths_and_values=paths_and_values
    )

    # Create a new configuration with the modified data
    return await canopy.create_config(
        session=session,
        config_type=config_type,
        name=name_of_config_to_be_created,
        config_data=new_config_data,
        sim_version=sim_version
    )


def get_config_data_derived_from_base_config_data_after_copying_from_paths_in_different_config_data(
    base_config_data: Dict[str, Any],
    copied_config_data: Dict[str, Any],
    paths_to_be_copied: List[str]
):
    """Get a new configuration after copying some paths with a different configuration.

    Args:
        session: Authenticated Canopy session object
        base_config_data: Base configuration data to modify
        copied_config_data: Copied configuration data to modify
        paths_to_be_copied: List of paths to be copied
        name_of_config_to_be_created: Name of the new configuration to be created
        config_type: Configuration type
        sim_version: Simulation version
        """
    # Check if base_config_data and copied_config_data are None
    if base_config_data is None or copied_config_data is None:
        raise ValueError("Base configuration data or copied configuration data is None. Please provide valid configuration data.")
    # Check if paths_to_be_copied is None
    if paths_to_be_copied is None:
        raise ValueError("Paths to be copied is None. Please provide valid paths.")
    
    # Create a paths and values dict to hold the paths and their new values from the swapped config
    paths_and_values = {}
    # Loop through the paths and set the new values from the swapped config
    for path in paths_to_be_copied:
        # Get the value at the specified path in the base config data
        current_value = get_value_at_path_in_config_data(base_config_data, path)
        if current_value is None:
            raise ValueError(f"Path '{path}' not found in base configuration data.")
        # Get the value at the specified path in the swapped config data
        new_value = get_value_at_path_in_config_data(copied_config_data, path)
        if new_value is None:
            raise ValueError(f"Path '{path}' not found in swapped configuration data.")
        # Set the new value at the specified path
        paths_and_values[path] = new_value

    # return the base config data with the swapped config data
    return get_config_data_derived_from_base_config_data_after_modifying_paths_with_new_values(
        base_config_data=base_config_data,
        paths_and_values=paths_and_values
    )
    

async def get_config_id_derived_from_base_config_data_after_copying_from_paths_in_different_config_data(
        session: Session,
        base_config_data: Dict[str, Any],
        copied_config_data: Dict[str, Any],
        paths_to_be_copied: List[str],
        name_of_config_to_be_created: str,
        config_type: str,
        sim_version: str
) -> str:
    """Get a new configuration after copying some paths with a different configuration.

    Args:
        session: Authenticated Canopy session object
        base_config_data: Base configuration data to modify
        copied_config_data: Copied configuration data to modify
        paths_to_be_copied: List of paths to be copied
        name_of_config_to_be_created: Name of the new configuration to be created
        config_type: Configuration type
        sim_version: Simulation version
    """
    # Get the base config data
    new_config_data = get_config_data_derived_from_base_config_data_after_copying_from_paths_in_different_config_data(
        base_config_data=base_config_data,
        copied_config_data=copied_config_data,
        paths_to_be_copied=paths_to_be_copied
    )

    # Create a new configuration with the modified data
    return await canopy.create_config(
        session=session,
        config_type=config_type,
        name=name_of_config_to_be_created,
        config_data=new_config_data,
        sim_version=sim_version
    )


async def get_study_id_in_worksheet_row(
        session: Session,
        worksheet_id: str,
        worksheet_row_name: str
) -> str:
    """Get the study ID in a worksheet row.

    Args:
        session: Authenticated Canopy session object
        worksheet_id: ID of the target worksheet
        worksheet_row_name: Worksheet row name to analyse
    """
    # Get the worksheet row
    worksheet_row = await get_worksheet_row_with_name_in_worksheet_with_id(
        session=session,
        worksheet_id=worksheet_id,
        worksheet_row_name=worksheet_row_name
    )

    # Check if the worksheet row exists
    b_worksheet_study_exists = await check_worksheet_row_study_exists(worksheet_row)
    if b_worksheet_study_exists is False:
        # Print a message if the study does not exist and return None
        print(f"Worksheet row '{worksheet_row_name}' does not have a study reference.")
        return None
    
    # Get the study ID
    study_id = worksheet_row.study.reference.target_id

    return study_id


async def get_stats_of_study_with_id(
        session: Session,
        study_id: str
):
    """Print the stats of a study with a given ID.

    Args:
        session: Authenticated Canopy session object
        study_id: ID of the target study
    """
    # Load the study
    study = await canopy.load_study(session, study_id, include_study_full_document=True)

    stats = {
        "study_id": study.document.document_id,
        "study_name": study.document.name,
        "study_type": study.document.data["studyType"],
        "study_state": study.document.data["studyState"],
        "simulation_count": study.simulation_count,
        "succeeded_simulation_count": study.succeeded_simulation_count
    }

    return stats


async def run_worksheet_row_study_with_configs_having_ids(
    session: Session,
    worksheet_id: str,
    source_row_name: str,
    sim_version: str,
    row_suffix: str,
    new_row_name: str = None,
    config_ids: List[str] = [],
    excluded_config_types: List[str] = [],
    sim_types: List[str] = ["DynamicLap"],
    study_type: str = "dynamicLap",
    notes: str = ""
):
    """Run a study with new configurations in a worksheet row.

    Args:
        session: Authenticated Canopy session object
        worksheet_id: ID of the target worksheet
        source_row_name: Worksheet row name used as the source for configs to be modified
        sim_version: Simulation version
        row_suffix: Suffix to append to modified row names
        config_ids: List of configuration IDs to execute in the new study
        excluded_config_types: List of configuration types to exclude from the new study
        sim_types: List of simulation types to run
        study_type: Type of study to run
        notes: Notes to add to the study
        """

    # Placeholder for the result
    result: Dict[str, str] = {}

    # Authenticate and initialize APIs
    session.authentication.authenticate()
    tenant_id = session.authentication.tenant_id
    worksheet_api = WorksheetApi(session.async_client)

    # Get worksheet data
    worksheet_result = await worksheet_api.worksheet_get_worksheet(tenant_id, worksheet_id)
    worksheet = worksheet_result.worksheet
    
    # If multiple rows with the same name exist, throw an error
    if len([row for row in worksheet.outline.rows if row.name == source_row_name]) > 1:
        raise ValueError(f"Multiple rows with the name '{source_row_name}' found in worksheet with ID '{worksheet_id}'. Please provide a unique row name.")
    
    # Loop through the worksheet rows and get the source row, throw an error if not found
    source_row = next((row for row in worksheet.outline.rows if row.name == source_row_name), None)
    if source_row is None:
        raise ValueError(f"Worksheet row '{source_row_name}' not found in worksheet with ID '{worksheet_id}'.")

    # Get the new row name
    if new_row_name is None:
        # If no new row name is provided, use the source row name with the suffix
        new_row_name = f"{source_row.name} {row_suffix}"

    # Get the new study name
    new_study_name = new_row_name
        
    # Create a placeholder for the new configs
    new_configs_list = []

    # Loop through the config_ids and load the configs
    for config_id in config_ids:
        # Load the new config
        new_config = await canopy.load_config(session, config_id)
        # Create a new worksheet config for the new config
        new_worksheet_config = WorksheetConfig(
            config_type=new_config.document.sub_type,
            reference=WorksheetConfigReference(
                tenant=ConfigReferenceTenant(
                    tenant_id=session.authentication.tenant_id,
                    target_id=config_id
                )
            ),
            inherit_reference=False
        )
        new_configs_list.append(new_worksheet_config)

    # Go through the source row configs and add them if the type is not in any of the config objects in new_configs_list
    for config in source_row.configs:
        # Check if the config type is already in the new configs list
        if config.config_type not in [c.config_type for c in new_configs_list]:
            # Load the source config
            loaded_config = await canopy.load_config(session, config.reference.tenant.target_id)
            # Create a new worksheet config for the source config
            new_worksheet_config = WorksheetConfig(
                config_type=loaded_config.document.sub_type,
                reference=WorksheetConfigReference(
                    tenant=ConfigReferenceTenant(
                        tenant_id=session.authentication.tenant_id,
                        target_id=config.reference.tenant.target_id
                    )
                ),
                inherit_reference=False
            )
            new_configs_list.append(new_worksheet_config)

    # Check if there are no type duplicates in the new configs
    config_types = [config.config_type for config in new_configs_list]
    if len(config_types) != len(set(config_types)):
        raise ValueError(f"Duplicate configuration types found in the new configs: {config_types}.")
    
    # Check if there are any excluded config types in the new configs and remove them
    for config in new_configs_list:
        if config.config_type in excluded_config_types:
            new_configs_list.remove(config)
    
    # Build study configuration
    sim_config: Dict[str, Any] = {}
    sources: List[NewStudyDataSource] = []

    # Process non-exploration configs
    for config in new_configs_list:
        loaded_config = await canopy.load_config(session, config.reference.tenant.target_id)
        sources.append(NewStudyDataSource(
            config_type=config.config_type,
            user_id=loaded_config.document.user_id,
            config_id=config.reference.tenant.target_id,
            name=loaded_config.document.name
        ))
        # Add to sim_config only if the config type is not exploration
        if config.config_type != "exploration":
            sim_config[config.config_type] = loaded_config.document.data

    # Create study configuration
    study = {
        'simTypes': sim_types,
        'simConfig': sim_config
    }

    # If a config of type exploration is in the new configs list, add it to the study dict
    exploration_config = next((config for config in new_configs_list if config.config_type == "exploration"), None)
    if exploration_config is not None:
        # Load the exploration config
        loaded_exploration_config = await canopy.load_config(session, exploration_config.reference.tenant.target_id)
        # Add exploration data to the study configuration
        study["exploration"] = loaded_exploration_config.document.data

    # Get the study API
    study_api = StudyApi(session.async_client)

    # Get the tenant ID
    tenant_id = session.authentication.tenant_id

    # Create study
    study_result = await study_api.study_post_study(
        tenant_id,
        StudyPostStudyRequest(
            name=new_study_name,
            study=study,
            is_transient=False,
            study_type=study_type,
            sources=sources,
            notes=notes,
            sim_version=sim_version
        )
    )

    # Create a new row with the new configs and study reference
    new_row = WorksheetRow(
        name=new_row_name,
        configs=new_configs_list,
        study=WorksheetRowStudy(
            reference=WorksheetStudyReference(
                tenant_id=tenant_id,
                target_id=study_result.study_id
            )
        )
    )

    # Update worksheet outline
    worksheet.outline.rows.append(new_row)
    
    # Commit changes
    worksheet_result = await worksheet_api.worksheet_put_worksheet(
        tenant_id,
        worksheet_id,
        WorksheetPutWorksheetRequest(
            name=worksheet.name,
            properties=worksheet.properties,
            outline=worksheet.outline,
            notes=worksheet.notes
        )
    )

    # Return the study ID
    return study_result.study_id


async def get_scalar_results_with_given_ids_from_jobs_in_study_with_id(
        session: Session,
        study_id: str,
        sim_type: str,
        scalar_ids: List[str],
        excluded_job_ids: List[str] = []
) -> List[Dict[str, List[Dict[str, Any]]]]:
    """Get scalar results from jobs in a study.

    Args:
        session: Authenticated Canopy session object
        study_id: ID of the study to analyse
        study_result: StudyResult object to check
        sim_type: Simulation type to check
        scalar_ids: List of scalar IDs to check against
        excluded_job_ids: List of job IDs to exclude from the search

    Returns:
        List of scalar results for the specified job IDs that looks like:
        [
            {
                'job_id': job_id,
                'inputs': [
                    { 'path': path_to_parameter,
                      'value': value_of_parameter
                    }
                ],
                'outputs': [
                    { 'id': scalar_id,
                      'value': scalar_value
                    }
                ]
            },
            ...
        ]

    """

    # Load the study with scalar data
    study_result = await canopy.load_study(
        session=session, 
        study_id=study_id,
        sim_type=sim_type,
        include_study_full_document=True, 
        include_job_metadata=True, 
        include_job_scalar_results=True, 
        include_job_vector_metadata=True
    )

    study_scalar_results = []
    # Loop through the jobs and get the scalar values
    for job in study_result.jobs:
        # Only process jobs that are not in the excluded_job_ids list
        if job.document.document_id not in excluded_job_ids:
            # Check if the scalar data is not empty
            if job.scalar_data is not None:
                # Loop through the scalar ids list and get the scalar values
                for scalar_id in scalar_ids:
                    # Check if the scalar exists in the job's scalar data
                    if scalar_id in job.scalar_data:
                        # If the job ID is not in the list of dicts in the study_scalar_results, add it and the relevant changes to the inputs
                        if job.document.document_id not in [s['job_id'] for s in study_scalar_results]:
                            # Get the changes from the job's result
                            changes = job.result.study_job.data.get('changes', [])
                            # Check if the changes exist
                            if changes:
                                inputs = [
                                    {
                                        "path": ivar['path'],
                                        "value": float(ivar['value'])
                                    } for ivar in changes
                                ]
                            else:
                                inputs = []
                            # Add the job ID and the relevant changes to the inputs
                            study_scalar_results.append(
                                {
                                    'job_id': job.document.document_id,
                                    'inputs': inputs,
                                    'outputs': []
                                }
                            )                   
                        # Get the scalar value
                        scalar_value = job.scalar_data.get(scalar_id)
                        # Add the scalar value to the scalar value dict
                        for s in study_scalar_results:
                            if s['job_id'] == job.document.document_id:
                                # Add the scalar value to the outputs list
                                s['outputs'].append(
                                    {
                                        'id': scalar_id,
                                        'value': float(scalar_value)
                                    }
                                )

    return study_scalar_results


async def get_scalar_data_from_jobs_in_study_with_id(
        session: Session,
        study_id: str,
        sim_type: str,
        scalar_limits: List[Dict[str, Any]] = []
) -> Dict[str, List[Dict[str, Any]]]:
    """Get scalar values from jobs in a study.

    Args:
        session: Authenticated Canopy session object
        study_id: ID of the study to analyse
        sim_type: Simulation type to check
        scalar_limits: List of scalar limits to check against
    """
    
    # Load the study with scalar data
    study = await canopy.load_study(
        session=session, 
        study_id=study_id,
        sim_type=sim_type,
        include_study_full_document=True, 
        include_job_metadata=True, 
        include_job_scalar_results=True, 
        include_job_vector_metadata=True
    )

    # Create a list of scalar values to check against the limits
    scalar_values: Dict[str, List[Dict[str, Any]]] = {}
    # Loop through the jobs and get the scalar values
    for job in study.jobs:
        # Initialize the scalar values dict for this job
        scalar_values[job.document.document_id] = []
        # Loop through the scalar limits list and get the scalar values
        for limit in scalar_limits:
            # If the limit has a 'min', then use it or set to -inf if not provided
            min_limit = limit.get('min', -np.inf)
            # If the limit has a 'max', then use it or set to inf if not provided
            max_limit = limit.get('max', np.inf)  
            # Check if the scalar exists in the job's scalar data and print a warning if not
            if limit['id'] not in job.scalar_data:
                print(f"Warning: Scalar '{limit['id']}' not found in job {job.document.document_id}.")
                continue
            else:
                # Get the scalar value
                scalar_value = job.scalar_data.get(limit['id'])
                # Add the scalar value, min and max limits to the scalar value dict
                scalar_values[job.document.document_id].append(
                    {
                        'scalar': limit['id'],
                        'scalar_value': float(scalar_value),
                        'min_limit': min_limit,
                        'max_limit': max_limit
                    }
                )

    return scalar_values


def get_disqualified_jobs_data_in_study_with_id_due_to_scalar_limit_violations(
    scalar_data: Dict[str, List[Dict[str, Any]]]
) -> Dict[str, List[Dict[str, Any]]]:
    """Get disqualified jobs in a study due to scalar limit violations.

    Args:
        scalar_data: Dictionary of scalar data for each job in the study
        """
    
    # Create a dictionary to hold the disqualified jobs
    disqualified_jobs: Dict[str, List[Dict[str, Any]]] = {}

    # Loop through the scalar data and check for limit violations
    for job_id, scalars in scalar_data.items():
        # Loop through the scalars and check for limit violations
        for scalar in scalars:
            # Check if the scalar value is outside the limits
            if 'min_limit' in scalar or 'max_limit' in scalar:
                if scalar['scalar_value'] < scalar['min_limit'] or scalar['scalar_value'] > scalar['max_limit']:
                    #If job_id is not in the disqualified jobs dict, add it
                    if job_id not in disqualified_jobs:
                        disqualified_jobs[job_id] = []
                    # Add the scalar to the disqualified jobs dict
                    disqualified_jobs[job_id].append(
                        {
                            'scalar': scalar['scalar'],
                            'scalar_value': scalar['scalar_value'],
                            'min_limit': scalar.get('min_limit', None),
                            'max_limit': scalar.get('max_limit', None)
                        }
                    )

    return disqualified_jobs
    

async def get_job_with_minimum_scalar_value(
        session: Session,
        study_id: str,
        sim_type: str,
        scalar_name: str,
        excluded_job_ids: List[str] = []
):
    """Get the job with the minimum scalar value in a study.

    Args:
        session: Authenticated Canopy session object
        study_id: ID of the study to analyse
        sim_type: Simulation type to check
        scalar_name: Name of the scalar to check
        exclude_job_ids: List of job IDs to exclude from the search

    Returns:
        JobResult object with the minimum scalar value
    """
    
    # Load the study with scalar data
    study = await canopy.load_study(
        session=session, 
        study_id=study_id,
        sim_type=sim_type,
        include_study_full_document=True, 
        include_job_metadata=True, 
        include_job_scalar_results=True, 
        include_job_vector_metadata=True
    )

    # Initialize variables to track the minimum scalar value and corresponding job
    min_scalar_value = float('inf')
    min_scalar_job_id = None

    # Loop through the jobs and get the scalar values
    for job in study.jobs:
        # Skip excluded jobs
        if job.document.document_id not in excluded_job_ids:
            print(f"Checking job {job.document.document_id} for scalar {scalar_name}...")
            # Check if the scalar exists in the job's scalar data
            if scalar_name in job.scalar_data:
                # Get the scalar value
                scalar_value = job.scalar_data.get(scalar_name)
                print(f"Job ID: {job.document.document_id}, Scalar Value: {scalar_value}")
                # Update minimum scalar value and corresponding job if necessary
                if float(scalar_value) < min_scalar_value:
                    min_scalar_value = float(scalar_value)
                    min_scalar_job_id = job.document.document_id
    
    return min_scalar_job_id, min_scalar_value


async def get_study_state(
        session: Session,
        study_id: str
    ) -> str:
    """Get the state of a study with a given ID.

    Args:
        session: Authenticated Canopy session object
        study_id: ID of the target study
    """
    # Load the study
    study = await canopy.load_study(
        session=session, 
        study_id=study_id, 
        include_study_full_document=True)
    
    if not study:
        return "Study not found"

    # Return the state of the study
    return study.document.data['studyState']


async def wait_for_simulation_complete(
    session,
    study_id: str,
    poll_interval: float = 2.0
) -> str:
    """Wait until the study is completed or failed."""
    while True:
        state = await get_study_state(session, study_id)
        if state not in ["running", "buildingAndRunning"]:
            return state
        await asyncio.sleep(poll_interval)


async def run_simulation_async(
    session,
    worksheet_id: str,
    base_row_name: str,
    parmeter_paths_and_values: Dict[str, Any],
    sim_version: str,
    scalar_ids: List[str],
    new_row_name: str="",
    sim_types: list[str] = ["DynamicLap"],
    study_type: str = "dynamicLap",
    excluded_job_ids: List[str] = [],
    poll_interval: float = 2.0
) -> Dict[str, Any]:
    """
    Runs simulation asynchronously in a new row of a worksheet, waits for completion, and returns scalar results.
    
    Args:
        session: Authenticated Canopy session
        worksheet_id: ID of the target worksheet
        base_row_name: Name of the base row to copy configurations from
        new_row_name: Name for the new row to be created
        parmeter_paths_and_values: Dictionary of parameter paths and their new values to modify in the base row's configurations
        sim_version: Simulation version to use
        scalar_ids: List of scalar IDs to retrieve from the simulation results
        sim_types: List of simulation types to run (default is ["DynamicLap"])
        study_type: Type of study to run (default is "dynamicLap")
        excluded_job_ids: List of job IDs to exclude from the results
        poll_interval: Time in seconds to wait between polling for simulation completion (default is 2.0 seconds)
        
    Returns:
        Dictionary of outputs keyed by scalar_id, and inputs for the job.
        If multiple jobs, returns a list of results (but only one if not excluded_job_ids).
    """

    base_row = await get_worksheet_row_with_name_in_worksheet_with_id(
        session=session,
        worksheet_id=worksheet_id,
        worksheet_row_name=base_row_name
    )

    # get all the config ids in the base row
    config_ids = [config.reference.tenant.target_id for config in base_row.configs]

    # print(f"Base row '{base_row_name}' has configs with IDs: {config_ids}")

    if not base_row:
        return None
    
    # Create a new worksheet row with the modified parameters based on the base row.
    modified_config_paths = {}

    for path, value in parmeter_paths_and_values.items():
        config_type = path.split('.')[0]
        config = next((c for c in base_row.configs if c.config_type == config_type), None)
        if not config:
            # print(f"Config with type '{config_type}' not found in base row '{base_row_name}'.")
            continue
        base_config_id = config.reference.tenant.target_id
        # Use config_type as the key in modified_config_paths
        if config_type not in modified_config_paths:
            modified_config_paths[config_type] = {"id": base_config_id}
        # Add the rest of the path as a key and the value
        modified_config_paths[config_type][path.split('.', 1)[1]] = value
    
    # Now modify the configs with the specified paths and values
    for config_type, config_data in modified_config_paths.items():
        loaded_config = await canopy.load_config(session, config_data['id'])
        current_data = loaded_config.document.data
        # print(f"Original data for config type '{config_type}':", current_data)
        # If you want to keep the original unmodified, use:
        # current_data = copy.deepcopy(loaded_config.document.data)
        for path, value in config_data.items():
            if path == 'id':
                continue
            keys = path.split('.')
            current_data_copy = current_data
            for key in keys[:-1]:
                if key in current_data_copy:
                    current_data_copy = current_data_copy[key]
                else:
                    print(f"Path '{path}' not found in base configuration data. Skipping modification.")
                    return None
            if keys[-1] in current_data_copy:
                current_data_copy[keys[-1]] = value
                # print(f"Modified path '{path}' to value '{value}' in config type '{config_type}'.")
            else:
                print(f"Path '{path}' not found in base configuration data. Skipping modification.")
                return None

        # Create a new configuration with the modified data
        new_config_id = await canopy.create_config(
            session=session,
            config_type=config_type,
            name=f"{loaded_config.document.name} {new_row_name}",
            config_data=current_data,
            sim_version=sim_version
        )

        # Replace the old config ID with the new one in config_ids
        for i, config_id in enumerate(config_ids):
            if config_id == base_config_id:
                config_ids[i] = new_config_id
                # print(f"Replaced config ID '{base_config_id}' with new ID '{new_config_id}'.")
    
    # Now run_worksheet_row_study_with_configs_having_ids
    study_id = await run_worksheet_row_study_with_configs_having_ids(
        session=session,
        worksheet_id=worksheet_id,
        source_row_name=base_row_name,
        sim_version=sim_version,
        row_suffix="",
        new_row_name=new_row_name,
        config_ids=config_ids,
        excluded_config_types=['exploration'],
        sim_types=sim_types,
        study_type=study_type
    )

    # Wait for simulation to complete
    state = await wait_for_simulation_complete(session, study_id, poll_interval)
    if state != "completed":
        print(f"Simulation failed with state: {state}")
        return None

    # Get scalar results for the job(s)
    return await get_scalar_results_with_given_ids_from_jobs_in_study_with_id(
        session=session,
        study_id=study_id,
        sim_type=sim_types[0], 
        scalar_ids=scalar_ids,
        excluded_job_ids=excluded_job_ids
    )


async def create_monte_carlo_exploration_config_from_swept_parameter_data(
        session: Session,
        name: str,
        sim_version: str,
        swept_parameter_data: List[Dict[str, Any]],
        n_monte_carlo_points: int = 1000
) -> str:
    """Create a Monte Carlo exploration config from swept parameter data.

    Args:
        session: Authenticated Canopy session object
        name: Name of the exploration config
        sim_version: Simulation version
        swept_parameter_data: List of dictionaries containing swept parameter data

    Returns:
        ID of the created exploration config
    """
    # Create the config data from the swept parameter data
    config_data = {
        "design": {
            "name": "Monte Carlo",
            "numberOfPoints": n_monte_carlo_points,
            "ranges": []
        }
    }

    for data in swept_parameter_data:
        # Check if the 'path', 'min' and 'max' keys are present in the data
        if 'path' not in data or 'min' not in data or 'max' not in data:
            raise ValueError(f"Missing required keys in swept parameter data: {data}")
        range_data = {
            "dimensionType": "interpolation",
            "distribution": "uniform",
            "parallelSubRanges": [
                {
                "parameterPath": data['path'],
                "valueType": "absolute",
                "valueStart": data['min'],
                "valueEnd": data['max'],
                "interpolationType": "linear"
                }
            ]
        }
        # Add the range data to the config data
        config_data['design']['ranges'].append(range_data)

    # Create a new exploration config
    exploration_config = await canopy.create_config(
        session=session,
        config_type="exploration",
        name=name,
        config_data=config_data,
        sim_version=sim_version
    )

    return exploration_config

## Authenticate

Run this to authenticate yourself with the canopy sims API. There will be a keyboard entry prompt displayed. 

In [196]:
# Enter sim_version
sim_version = '1.12720'
# Enter the worksheet row names from which we extract the car configurations from. These rows will not be modified.
source_row_names = [
    'base'
]

In [197]:
session = await authenticate_canopy_sims_api(canopy.prompt_for_authentication())
user_id = session.authentication.user_id

Authentication failed: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Authenticated successfully!


## Specify the sim version and starting row name

## Create a new worksheet if needed

Make a note of the returned worksheet id. Put your worksheet ID for the worksheet_id variable. This can be found in the URL of the worksheet: https://portal.canopysimulations.com/worksheets/client_id/worksheet_id. Or, use the id from the created worksheet.

In [ ]:
worksheet_id, study_id_0 = await create_worksheet_and_run_study(
    session=session,
    worksheet_name="My Worksheet",
    starting_row_name=source_row_names[0],
    sim_version=sim_version,
    default_car_name="Canopy F1 Car 2025 Race Engineering Challenge",
    default_track_name="Barcelona-F1",
    default_weather_name="25 deg, dry",
    default_user_maths="Canopy 2025 Race Engineering Challenge"
)

print(f"Created worksheet with ID: {worksheet_id} with study ID: {study_id_0}")

Created worksheet with ID: 4343bcdae11a4f60a70303f21dde39d0 with study ID: 497e6c767bb04164b5bc8f8dfe16f3bd


In [198]:
import asyncio
import optuna

# Configuration
PARAM_RANGES = {
    'car.aero.flapAngles.aFlapF': (0.40, 0.70),
    # 'car.chassis.carRunningMass.rWeightBalF': (0.450, 0.455),
    'car.chassis.hRideFSetup': (0.01, 0.10),
    'car.chassis.hRideRSetup': (0.01, 0.10),
    'car.suspension.front.external.aCamberSetupAlignment.aCamberSetup': (-0.15, 0.05),
    # 'car.suspension.front.internal.antiRollBar.kAntiRollBar': (0.0, 100000.0),
    # 'car.suspension.front.internal.torsionBar.kTorsionBar': (0.0, 10000.0),
    'car.suspension.rear.external.aCamberSetupAlignment.aCamberSetup': (-0.15, 0.05),
    # 'car.suspension.rear.internal.antiRollBar.kAntiRollBar': (0.0, 100000.0),
    # 'car.suspension.rear.internal.triSpring.kSpring': (0.0, 10000.0)
    }

# Turn CONSTRAINT_SPEC into a dict
CONSTRAINT_SPEC = {
    'RaceEng_EBottoming':(0.0, 0.5),
    'RaceEng_UndersteerT10Entry':(-4e-3, np.inf),
    'RaceEng_UndersteerT9':(-1.5e-3, np.inf),
    'RaceEng_aCamberFEOS':(-3.5, 1.0),
    'RaceEng_aCamberREOS':(-1.5, 1.0),
    'RaceEng_kHeaveF':(0.0, 850000),
    'RaceEng_kHeaveR':(0.0, 400000),
    'RaceEng_kRoll':(0.0, 350000),
    'RaceEng_rBrakeBalHighPressure':(0.6, np.inf),
    'tLapTotal': (0.0, 77.635)
}

class AsyncOptimizer:
    def __init__(
            self, 
            session: Session,
            param_ranges: Dict[str, tuple],
            constraint_spec: Dict[str, tuple],
            worksheet_id: str,
            base_row_name: str,
            sim_version: str,
            poll_interval: float = 2.0
            ):
        
        self.session = session
        self.param_ranges = param_ranges
        self.constraint_spec = constraint_spec
        self.worksheet_id = worksheet_id
        self.base_row_name = base_row_name  
        self.sim_version = sim_version
        self.poll_interval = poll_interval
        # Initialize current max laptime from constraints
        self.base_tLapTotal = self.constraint_spec.get('tLapTotal')[1]

    async def objective(self, trial):
        """Async objective function for Optuna trials"""
        # 1. Suggest parameters
        params = {}
        for param_name in self.param_ranges.keys():
            low, high = self.param_ranges.get(param_name)  # Default range if not specified
            params[param_name] = trial.suggest_float(param_name, low, high)
        
        # Generate unique row name using trial number
        new_row_name = f"Trial_{trial.number}"

        # Store row name in trial attributes
        trial.set_user_attr("row_name", new_row_name)
        
        # Start simulation
        results = await run_simulation_async(
            session=self.session,
            worksheet_id=self.worksheet_id,
            base_row_name=self.base_row_name,
            parmeter_paths_and_values=params,
            sim_version=self.sim_version,
            scalar_ids=[constraint for constraint in self.constraint_spec.keys()],
            new_row_name=new_row_name
        )

        if not results or len(results) == 0:
            raise optuna.TrialPruned("No results returned from simulation.")

        # Get the job result from the first job
        job_result = results[0]

        # Make sure 'outputs' is a list of dicts
        outputs_list = job_result['outputs']

        # Convert to a flat dict for easy access
        outputs = {item['id']: item['value'] for item in outputs_list}
        
        # 5. Calculate tLapTotal with penalty for constraints
        tLapTotal = outputs['tLapTotal']
        penalty = self._calculate_penalty(outputs)
        # tLapTotal should be penalised if it's greater than the base tLapTotal
        # Store raw values for potential constraint updates
        trial.set_user_attr("tLapTotal", tLapTotal)
        trial.set_user_attr("penalty", penalty)
        return penalty

    def _calculate_penalty(self, outputs: dict) -> float:
        """Apply penalty for constraint violations"""
        penalty = 0.0
        for constraint_id, constraint_bounds in self.constraint_spec.items():
            value = outputs.get(constraint_id, None)
            min_limit, max_limit = constraint_bounds
            if value is None:
                penalty += 1e9  # Missing data penalty
                continue
            constraint_min_limit_penalty = max(0, min_limit - value) * 1e6
            constraint_max_limit_penalty = max(0, value - max_limit) * 1e6
            print(f"Constraint '{constraint_id}': {value} (Min: {min_limit}, Max: {max_limit}, max_limit_penalty: {constraint_max_limit_penalty}, min_limit_penalty: {constraint_min_limit_penalty})")
            # If the value is outside the bounds, apply a penalty
            penalty += (constraint_min_limit_penalty + constraint_max_limit_penalty)
        return penalty

In [ ]:
study = optuna.create_study(
    direction='minimize',
)

optimizer = AsyncOptimizer(
    session=session,
    param_ranges=PARAM_RANGES,
    constraint_spec=CONSTRAINT_SPEC,
    worksheet_id=worksheet_id,
    base_row_name=source_row_names[0],
    sim_version=sim_version,
    poll_interval=15.0
)

# Starting laptime
current_tLapTotal = optimizer.base_tLapTotal

# Run optimization sequentially
n_trials = 100
for trial_num in range(n_trials):
    trial = study.ask()
    try:
        value = await optimizer.objective(trial)
        study.tell(trial, value)
        
        # Update laptime constraint if feasible solution found
        tLapTotal_from_trial = trial.user_attrs["tLapTotal"]
        if tLapTotal_from_trial < current_tLapTotal:
            current_tLapTotal = tLapTotal_from_trial * 1.01  # 1% buffer
            # Update the constraint spec
            for constraint_id, constraint_values in optimizer.constraint_spec.items():
                if constraint_id == 'tLapTotal':
                    optimizer.constraint_spec[constraint_id] = (0.0, current_tLapTotal)
                    print(f"Updated tLapTotal constraint to {current_tLapTotal:.4f}s")
        
        print(f"Completed trial {trial_num}/{n_trials} - Value: {value:.3f}")
    except Exception as e:
        study.tell(trial, float('inf'))
        print(f"Trial {trial_num} failed: {str(e)}")

# Output results
best_trial = study.best_trial
best_row_name = best_trial.user_attrs["row_name"]

print("\n=== OPTIMIZATION RESULTS ===")
print(f"Best laptime: {best_trial.value:.3f}s")
print(f"Best worksheet row: {best_row_name}")
print("\nBest parameters:")
for param, value in best_trial.params.items():
    print(f"  {param}: {value}")

[I 2025-07-01 19:17:41,437] A new study created in memory with name: no-name-2bd94e37-b2cf-4b2b-887f-a249d189ca6c
Retrying https://api.canopysimulations.com/worksheets/a4ed02d5506c4237a58d520e151f74be/4343bcdae11a4f60a70303f21dde39d0


Completed trial 0/100 - Value: 12570187.000


Retrying https://api.canopysimulations.com/studies/a4ed02d5506c4237a58d520e151f74be/72fee66f11b74c65b9a97623e95cb89e


Completed trial 1/100 - Value: 7853361.000
Simulation failed with state: failed
Trial 2 failed: No results returned from simulation.
Completed trial 3/100 - Value: 4868502.000
Completed trial 4/100 - Value: 15416385.000
Completed trial 5/100 - Value: 8150987.000
Completed trial 6/100 - Value: 14618636.000
Completed trial 7/100 - Value: 1113596.000
Completed trial 8/100 - Value: 6730854.000


Retrying https://api.canopysimulations.com/studies/a4ed02d5506c4237a58d520e151f74be/053f33b72cfc46389fe14445241b63c7


...

Retrying https://api.canopysimulations.com/studies/a4ed02d5506c4237a58d520e151f74be/62e3d751b6e84fcebff7daa001f1d1ec


Completed trial 19/100 - Value: 1666839.000
Completed trial 20/100 - Value: 2971668.000
Completed trial 21/100 - Value: 1026796.000
Completed trial 22/100 - Value: 1462992.000


Retrying https://api.canopysimulations.com/studies/a4ed02d5506c4237a58d520e151f74be/8a6eae159d1742d2ac1a74b8013a1e58


Trial 23 failed: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Trial 24 failed: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Trial 25 failed: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))


CancelledError: 

## Check if study with id is completed

Run the cell below id identify if all the jobs in the study have completed. If completed, proceed to the next step. Wait if not.

In [65]:
print(f"Study ID: {study_id_0} Study state: {await get_study_state(session=session, study_id=study_id_0)}")

print(f"Scalar results: {await get_scalar_results_with_given_ids_from_jobs_in_study_with_id(session=session, study_id=study_id_0, sim_type='DynamicLap', scalar_ids=constraint_ids)}")

Study ID: 846845cdb4dd45f3b3087170f52cb0fd Study state: completed
Scalar results: [{'job_id': '846845cdb4dd45f3b3087170f52cb0fd-0', 'inputs': [], 'outputs': [{'id': 'RaceEng_EBottoming', 'value': 0.04111}, {'id': 'RaceEng_UndersteerT10Entry', 'value': -0.002487}, {'id': 'RaceEng_UndersteerT9', 'value': -0.00134}, {'id': 'RaceEng_aCamberFEOS', 'value': -3.230183}, {'id': 'RaceEng_aCamberREOS', 'value': -0.845681}, {'id': 'RaceEng_kHeaveF', 'value': 826435.378929}, {'id': 'RaceEng_kHeaveR', 'value': 376095.494436}, {'id': 'RaceEng_kRoll', 'value': 325.237112}, {'id': 'RaceEng_rBrakeBalHighPressure', 'value': 0.630874}, {'id': 'tLapTotal', 'value': 77.594288}]}]


In [ ]:
async def run_simulation_async(
    simulation_id: str,
    check_state_func: callable,
    get_results_func: callable,
    poll_interval: float = 1.0
) -> dict:
    """
    Runs simulation asynchronously with state polling until completion.
    
    Args:
        simulation_id: Unique simulation identifier
        check_state_func: Coroutine to check simulation state (must be async)
        get_results_func: Coroutine to fetch results (must be async)
        poll_interval: Seconds between state checks
        
    Returns:
        Simulation results dictionary
    """
    while True:
        state = await check_state_func(simulation_id)
        if state == 'completed':
            return await get_results_func(simulation_id)
        await asyncio.sleep(poll_interval)

Populate the created or starting worksheet with some rows containing the configs of interest and run the study. We set the source row names of interest below, for example, here it's called 'base'. All the following studies will start off with these rows.

![alt text](images/RaceEngChallenge/image-11.png)

## Run this to reset the worksheet (WARNING: Running this will erase data from all rows expect those in the source_row_names list)

The following block can be called if you need to clean all the rows of the worksheet, except for those specified in the row_names list.

In [ ]:
#  First, reset the worksheet to only contain the rows we want to modify
reset_worksheet_result = await reset_worksheet(
    session=session,
    worksheet_id=worksheet_id,
    row_names=source_row_names
)

## Setup explorations

### Swept parameters and scalar limits

For this challenge, the objective is to identify the fastest legal car by varying certian parameters listed below, given a set of constraints. We will start with some fundamental parameters to be swept first.

The scalar limits must be adhered to by the identified car, else, the car is deemed to be illegal. Note that tLapTotal is also specified as a scalar limit with a max value of the lap time in the baseline configuration.. This is because it's quite a convenient way to introduce it into the function so that any cars slower than the baseline car are rejected.

Run the cell below to setup these parameters and limits.

In [ ]:
study_1_swept_param_paths = [
    'car.chassis.hRideFSetup',
    'car.chassis.hRideRSetup',
    'car.aero.flapAngles.aFlapF',
    'car.suspension.front.internal.antiRollBar.kAntiRollBar',
    'car.suspension.rear.internal.antiRollBar.kAntiRollBar',
    'car.suspension.front.internal.torsionBar.kTorsionBar',
    'car.suspension.rear.internal.triSpring.kSpring',
    'car.chassis.carRunningMass.rWeightBalF',
    'car.suspension.front.external.aCamberSetupAlignment.aCamberSetup',
    'car.suspension.rear.external.aCamberSetupAlignment.aCamberSetup',
    'car.powertrain.rearAxleTransmission.diff.MDiffDemandOutputs[0]',
    'car.powertrain.rearAxleTransmission.diff.MDiffDemandOutputs[2]',
    'car.control.brakeBalanceOptimisation.mapOutput.values[0]',
    'car.control.brakeBalanceOptimisation.mapOutput.values[1]',
    'car.control.brakeBalanceOptimisation.mapOutput.values[2]',
    'car.control.brakeBalanceOptimisation.mapOutput.values[3]',
    'car.control.brakeBalanceOptimisation.mapOutput.values[4]',
    'car.control.brakeBalanceOptimisation.mapOutput.values[5]',
    'car.control.brakeBalanceOptimisation.mapOutput.values[6]',
    'car.control.brakeBalanceOptimisation.mapOutput.values[7]',
    'car.control.brakeBalanceOptimisation.mapOutput.values[8]',
]

study_1_swept_parameter_data = [
    { 
        'id': 'hRideFSetup',
        'path': 'car.chassis.hRideFSetup',
        'min': 0.01,
        'max': 0.05
    },
    {
        'id': 'hRideRSetup',
        'path': 'car.chassis.hRideRSetup',
        'min': 0.01,
        'max': 0.08
    },
    {
        'id': 'aFlapF',
        'path': 'car.aero.flapAngles.aFlapF',
        'min': 0.4,
        'max': 1.0
    },
    {
        'id':'rWeightBalF',
        'path':'car.chassis.carRunningMass.rWeightBalF',
        'min': 0.45,
        'max': 0.47
    },
    {
        'id':'aCamberSetupF',
        'path':'car.suspension.front.external.aCamberSetupAlignment.aCamberSetup',
        'min': -0.1,
        'max': 0.0
    },
    {
        'id':'aCamberSetupR',
        'path':'car.suspension.rear.external.aCamberSetupAlignment.aCamberSetup',
        'min': -0.1,
        'max': 0.0
    },
    # {
    #     'id': 'kAntiRollBarF',
    #     'path': 'car.suspension.front.internal.antiRollBar.kAntiRollBar',
    #     'min': 0.0,
    #     'max': 100000.0
    # },
    # {
    #     'id': 'kAntiRollBarR',
    #     'path': 'car.suspension.rear.internal.antiRollBar.kAntiRollBar',
    #     'min': 0.0,
    #     'max': 100000.0
    # },
    # {
    #     'id': 'kTorsionBarF',
    #     'path': 'car.suspension.front.internal.torsionBar.kTorsionBar',
    #     'min': 0.0,
    #     'max': 10000.0
    # },
    # {
    #     'id': 'kTorsionBarR',
    #     'path': 'car.suspension.rear.internal.triSpring.kSpring',
    #     'min': 0.0,
    #     'max': 10000.0
    # }
]


study_1_dynamic_lap_scalar_limits = [
    {
        'id':'RaceEng_EBottoming',
        'max':0.5
    },
    {
        'id':'RaceEng_UndersteerT10Entry',
        'min':-4e-3,
    },
    {
        'id':'RaceEng_UndersteerT9',
        'min':-1.5e-3,
    },
    {
        'id':'RaceEng_aCamberFEOS',
        'min':-3.5,
        'max': 0.0
    },
    {
        'id':'RaceEng_aCamberREOS',
        'min':-1.5,
        'max': 0.0
    },
    {
        'id':'RaceEng_kHeaveF',
        'max':850000
    },
    {
        'id':'RaceEng_kHeaveR',
        'max':400000
    },
    {
        'id':'RaceEng_kRoll',
        'max':350000
    },
    {
        'id':'RaceEng_rBrakeBalHighPressure',
        'min':0.6
    },
    {
        'id':'tLapTotal',
        'min':0.0,
        'max':77.589
    }
]



## Check the scalar limits of the baseline study

Now, check the baseline study in the worksheet by retrieving the job scalar data and identify if the baseline car is legal. As seen, the baseline car is legal.

In [ ]:
scalar_data = await get_scalar_data_from_jobs_in_study_with_id(
    session=session,
    study_id=study_id_0,
    sim_type='DynamicLap',
    scalar_limits=study_1_dynamic_lap_scalar_limits
)

# print(scalar_data)

disqualified_jobs_data = get_disqualified_jobs_data_in_study_with_id_due_to_scalar_limit_violations(
    scalar_data=scalar_data
)

print(f"Number of disqualified jobs: {len(disqualified_jobs_data)}")

## Run new study with a desired exploration

Let's try a minimal heurisitcs incorporated approach to identify a faster car. One of the best ways to achieve this is to run some monte carlo explorations, whch we can setup and run the worksheet quite easily, following an approach shown in the cell below.

In [ ]:
# Create a Monte Carlo exploration config
exploration_config_id_0 = await create_monte_carlo_exploration_config_from_swept_parameter_data(
    session=session,
    name='exploration_mc_0',
    sim_version=sim_version,
    swept_parameter_data=study_1_swept_parameter_data,
    n_monte_carlo_points=1000
)

# print(exploration_config_id_0)

study_id_1 = await run_worksheet_row_study_with_configs_having_ids(
    session=session,
    worksheet_id=worksheet_id,
    source_row_name='base',
    sim_version=sim_version,
    row_suffix='exploration_mc_0',
    config_ids=[exploration_config_id_0]
)

You should see the new row created with the study running after refreshing the browser.

![alt text](images/RaceEngChallenge/image-12.png)

## Check if study with id is completed

Run the cell below id identify if all the jobs in the study have completed. If completed, proceed to the next step.

In [ ]:
print(f"Study ID: {study_id_1}")

study_1 = await canopy.load_study(
    session=session,
    study_id=study_id_1,
)

print(f"study_state: {study_1.document.data['studyState']}")

## Re-check the scalar limits

The cell below loads the scalar data of all the jobs in the study and can take some time, especially for a Monte Carlo exploration with 100s of sims.

The warnings that are displayed are caused by jobs which have failed.

![alt text](images/RaceEngChallenge/image-6.png)

![alt text](images/RaceEngChallenge/image-7.png)

In [ ]:
study_1_scalar_data = await get_scalar_data_from_jobs_in_study_with_id(
    session=session,
    study_id=study_id_1,
    sim_type='DynamicLap',
    scalar_limits=study_1_dynamic_lap_scalar_limits
)

# print(scalar_data)

## Check the disqualified jobs

Run the cell below to get the disqualified jobs which will be needed to identify the fastest car from the qualified jobs in the next step.

In [ ]:
study_1_disqualified_jobs_data = get_disqualified_jobs_data_in_study_with_id_due_to_scalar_limit_violations(
    scalar_data=study_1_scalar_data
)

print(study_1_disqualified_jobs_data)

#Get the number of disqualified jobs
study_1_disqualified_jobs_count = len(study_1_disqualified_jobs_data.keys())
print(f"Number of disqualified jobs: {study_1_disqualified_jobs_count}")

## Get the job with the minimum tLapTotal

Run the cell below to idnetify the fastest car from the lot, if there is any. And we have managed to find a legal car with a laptime of 77.5707s.

![alt text](images/RaceEngChallenge/image-8.png)

In [ ]:
# Create a list of job_ids to exclude from the analysis from the keys of disqualified_jobs_data
study_1_excluded_job_ids = list(study_1_disqualified_jobs_data.keys())

study_1_results = await get_job_with_minimum_scalar_value(
    session=session,
    study_id=study_id_1,
    sim_type='DynamicLap',
    scalar_name='tLapTotal',
    excluded_job_ids=study_1_excluded_job_ids
)

print(study_1_results)

Update the upper bound of tLapTotal limit with the new lap time.

In [ ]:
study_2_dynamic_lap_scalar_limits = [
    {
        'id':'RaceEng_EBottoming',
        'max':0.5
    },
    {
        'id':'RaceEng_UndersteerT10Entry',
        'min':-4e-3,
    },
    {
        'id':'RaceEng_UndersteerT9',
        'min':-1.5e-3,
    },
    {
        'id':'RaceEng_aCamberFEOS',
        'min':-3.5,
        'max': 0.0
    },
    {
        'id':'RaceEng_aCamberREOS',
        'min':-1.5,
        'max': 0.0
    },
    {
        'id':'RaceEng_kHeaveF',
        'max':850000
    },
    {
        'id':'RaceEng_kHeaveR',
        'max':400000
    },
    {
        'id':'RaceEng_kRoll',
        'max':350000
    },
    {
        'id':'RaceEng_rBrakeBalHighPressure',
        'min':0.6
    },
    {
        'id':'tLapTotal',
        'min':0.0,
        'max':77.5707
    }
]

## Re-run the sudy with a new exploration

Now, the question of how to identify which numbers to use for the next iteration comes up. 

If you want to fully automate the process, this should be possible by writing up a fancy algorithm of your choosing and calling these functions iteratively until you hit a limit.

Or, you can go quick and dirty by taking a look at the cross section plots and visually observing the trend of tLapTotal:DynamicLap with each swept parameter.

![alt text](images/RaceEngChallenge/image-9.png)

In [ ]:
study_2_swept_parameter_data = [
    { 
        'id': 'hRideFSetup',
        'path': 'car.chassis.hRideFSetup',
        'min': 0.022,
        'max': 0.026
    },
    {
        'id': 'hRideRSetup',
        'path': 'car.chassis.hRideRSetup',
        'min': 0.045,
        'max': 0.055
    },
    {
        'id': 'aFlapF',
        'path': 'car.aero.flapAngles.aFlapF',
        'min': 0.525,
        'max': 0.575
    },
    {
        'id':'rWeightBalF',
        'path':'car.chassis.carRunningMass.rWeightBalF',
        'min': 0.451,
        'max': 0.453
    },
    {
        'id':'aCamberSetupF',
        'path':'car.suspension.front.external.aCamberSetupAlignment.aCamberSetup',
        'min': -0.12,
        'max': -0.09
    },
    {
        'id':'aCamberSetupR',
        'path':'car.suspension.rear.external.aCamberSetupAlignment.aCamberSetup',
        'min': -0.10,
        'max': -0.08
    },
    # {
    #     'id': 'kAntiRollBarF',
    #     'path': 'car.suspension.front.internal.antiRollBar.kAntiRollBar',
    #     'min': 0.0,
    #     'max': 100000.0
    # },
    # {
    #     'id': 'kAntiRollBarR',
    #     'path': 'car.suspension.rear.internal.antiRollBar.kAntiRollBar',
    #     'min': 0.0,
    #     'max': 100000.0
    # },
    # {
    #     'id': 'kTorsionBarF',
    #     'path': 'car.suspension.front.internal.torsionBar.kTorsionBar',
    #     'min': 0.0,
    #     'max': 10000.0
    # },
    # {
    #     'id': 'kTorsionBarR',
    #     'path': 'car.suspension.rear.internal.triSpring.kSpring',
    #     'min': 0.0,
    #     'max': 10000.0
    # }
]

## Run study with new exploration

Let's call this exploration_mc_1

In [ ]:
# Create a Monte Carlo exploration config
exploration_config_id_1 = await create_monte_carlo_exploration_config_from_swept_parameter_data(
    session=session,
    name='exploration_mc_1',
    sim_version=sim_version,
    swept_parameter_data=study_2_swept_parameter_data,
    n_monte_carlo_points=1000
)

# print(exploration_config_id)

study_id_2 = await run_worksheet_row_study_with_configs_having_ids(
    session=session,
    worksheet_id=worksheet_id,
    source_row_name='base',
    sim_version=sim_version,
    row_suffix='exploration_mc_1',
    config_ids=[exploration_config_id_1]
)

# Check if study with id is completed

In [ ]:
study_2 = await canopy.load_study(
    session=session,
    study_id=study_id_2,
)

print(f"study_state: {study_2.document.data['studyState']}")

## Re-check the scalar limits

In [ ]:
study_2_scalar_data = await get_scalar_data_from_jobs_in_study_with_id(
    session=session,
    study_id=study_id_2,
    sim_type='DynamicLap',
    scalar_limits=study_2_dynamic_lap_scalar_limits
)

# print(scalar_data)

## Check the disqualified jobs

In [ ]:
study_2_disqualified_jobs_data = get_disqualified_jobs_data_in_study_with_id_due_to_scalar_limit_violations(
    scalar_data=study_2_scalar_data
)

print(study_2_disqualified_jobs_data)

#Get the number of disqualified jobs
study_2_disqualified_jobs_count = len(study_2_disqualified_jobs_data.keys())
print(f"Number of disqualified jobs: {study_2_disqualified_jobs_count}")

## Get the job with the minimum tLapTotal

In [ ]:
# Create a list of job_ids to exclude from the analysis from the keys of disqualified_jobs_data
study_2_excluded_job_ids = list(study_2_disqualified_jobs_data.keys())

study_2_results = await get_job_with_minimum_scalar_value(
    session=session,
    study_id=study_id_2,
    sim_type='DynamicLap',
    scalar_name='tLapTotal',
    excluded_job_ids=study_2_excluded_job_ids
)

print(study_2_results)

## Re-iterate wth other parameter sweeps

Since no faster legal car was found, you an re-iterate using a different set of parameter sweeps.